In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/earthquake-in-japan/japan_2000_2023_query.csv
/kaggle/input/earthquake-in-japan/gadm41_JPN_0.shx
/kaggle/input/earthquake-in-japan/gadm41_JPN_2.json
/kaggle/input/earthquake-in-japan/morocco_20230908_query.csv
/kaggle/input/earthquake-in-japan/PastHugeEarthquakeinNankai.csv
/kaggle/input/earthquake-in-japan/ishikawa_202401_query.csv
/kaggle/input/earthquake-in-japan/japan-20190101_20211203_query.csv
/kaggle/input/earthquake-in-japan/japan_20220319_query.csv
/kaggle/input/earthquake-in-japan/gadm41_JPN_1.json
/kaggle/input/earthquake-in-japan/japan_20220317_query.csv
/kaggle/input/earthquake-in-japan/ishikawa_202401.json
/kaggle/input/earthquake-in-japan/gadm41_JPN_0.shp
/kaggle/input/earthquake-in-japan/tonga_20210101_20220117_query.csv
/kaggle/input/earthquake-in-japan/japan_20190101-20211009_query.csv
/kaggle/input/earthquake-in-japan/afghanistan_20231008_query.csv
/kaggle/input/earthquake-in-japan/japan_2023-.json
/kaggle/input/earthquake-in-japan/afghanistan_20231015_

In [2]:
earthquakes = pd.read_csv('/kaggle/input/earthquake-in-japan/japan_2000_2023_query.csv')
earthquakes = pd.DataFrame(earthquakes)
earthquakes=earthquakes[earthquakes['place'].str.contains('Japan',na=False)]
earthquakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-03-02T07:19:01.487Z,32.5313,135.7405,16.123,4.5,mb,30.0,95.0,1.970,0.44,...,2023-03-02T08:13:16.040Z,"135 km S of Shing?, Japan",earthquake,5.07,4.815,0.131,18.0,reviewed,us,us
1,2023-03-02T06:47:39.954Z,33.3315,139.4942,9.945,5.1,mww,65.0,77.0,0.344,0.49,...,2023-03-02T07:07:30.040Z,"157 km SSE of Shimoda, Japan",earthquake,5.01,4.540,0.073,18.0,reviewed,us,us
3,2023-02-28T15:48:02.842Z,35.6762,140.7823,43.894,4.5,mwr,27.0,139.0,2.261,0.65,...,2023-02-28T16:49:02.040Z,"7 km SW of Hasaki, Japan",earthquake,6.74,5.915,0.089,12.0,reviewed,us,us
4,2023-02-28T11:12:09.077Z,32.9764,141.6939,10.000,5.2,mww,136.0,93.0,1.585,0.79,...,2023-02-28T11:32:11.040Z,"272 km SSE of Katsuura, Japan",earthquake,6.71,1.847,0.083,14.0,reviewed,us,us
5,2023-02-28T03:55:10.857Z,35.8699,140.9309,46.248,4.7,mb,55.0,128.0,2.301,0.59,...,2023-03-01T04:02:27.040Z,"17 km NNE of Hasaki, Japan",earthquake,2.49,6.671,0.066,70.0,reviewed,us,us


In [3]:
earthquakes = earthquakes.loc[:,['mag','magType','dmin','place']]
earthquakes = earthquakes.dropna(subset='place')
earthquakes

,mag,magType,dmin,place
0,4.5,mb,1.970,"135 km S of Shing?, Japan"
1,5.1,mww,0.344,"157 km SSE of Shimoda, Japan"
3,4.5,mwr,2.261,"7 km SW of Hasaki, Japan"
4,5.2,mww,1.585,"272 km SSE of Katsuura, Japan"
5,4.7,mb,2.301,"17 km NNE of Hasaki, Japan"
...,...,...,...,...
11127,4.8,mb,NaN,"191 km E of Hachinohe, Japan"
11128,4.9,mb,NaN,"8 km E of Hitachi-Naka, Japan"
11129,5.4,mwc,NaN,"50 km ESE of Namie, Japan"
11130,4.5,mb,NaN,"43 km SSW of Nemuro, Japan"


In [4]:
earthquakes.rename(columns={'place':'city'},inplace=True)
earthquakes.head()

,mag,magType,dmin,city
0,4.5,mb,1.970,"135 km S of Shing?, Japan"
1,5.1,mww,0.344,"157 km SSE of Shimoda, Japan"
3,4.5,mwr,2.261,"7 km SW of Hasaki, Japan"
4,5.2,mww,1.585,"272 km SSE of Katsuura, Japan"
5,4.7,mb,2.301,"17 km NNE of Hasaki, Japan"


In [5]:
earthquakes['city'] = earthquakes['city'].str.extract(r'\bof\s+(\w+)\s*,', expand=False)

In [6]:
earthquakes = earthquakes.dropna(subset='city')
earthquakes


,mag,magType,dmin,city
1,5.1,mww,0.344,Shimoda
3,4.5,mwr,2.261,Hasaki
4,5.2,mww,1.585,Katsuura
5,4.7,mb,2.301,Hasaki
6,4.6,mb,2.355,Honshu
...,...,...,...,...
11126,4.8,mb,NaN,Ishinomaki
11127,4.8,mb,NaN,Hachinohe
11129,5.4,mwc,NaN,Namie
11130,4.5,mb,NaN,Nemuro


In [7]:
earthquakes['city'].value_counts()

city
Namie         956
Ishinomaki    803
Hasaki        768
Iwaki         593
Miyako        517
             ... 
Kobayashi       1
Numata          1
Tsubata         1
Nagasaki        1
Aomori          1
Name: count, Length: 301, dtype: int64

In [8]:
earthquakes.describe()

,mag,dmin
count,8017.000000,2499.000000
mean,4.818493,1.739376
std,0.378112,1.079582
min,4.500000,0.038000
25%,4.600000,0.949500
50%,4.700000,1.532000
75%,4.900000,2.389000
max,8.160000,19.725000
